# Examples and Tasks with Pandas

Pandas is another Python library that provides great functions to work with data. Though being very handy it is not exactly fast. So be aware when working with large datasets - this is going to be slow with Pandas. However for data exploration and testing of functions Pandas is super helpful.

## Loading data from files

The following three examples show, how to load response time data from three different file types.

In [ ]:
import pandas as pd

data = pd.read_csv("data-samples/responsetimes.csv")
data

In [ ]:
import pandas as pd

data = pd.read_excel("data-samples/responsetimes.xlsx")
data

In [ ]:
import pandas as pd

data = pd.read_excel("data-samples/responsetimes.ods", engine="odf")
data

## Connecting to data bases

As we put data into Timescale, let's connect and see if Pandas can work with query results too.

In [19]:
import psycopg2
import logging
from psycopg2.extras import LoggingConnection

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

db_settings = {
    "user": "postgres",
    "password": "password",
    "host": "localhost",
    "database": "sampledata",
}

conn = psycopg2.connect(connection_factory=LoggingConnection, **db_settings)
conn.initialize(logger)
cursor = conn.cursor()
# use the cursor to interact with your database
cursor.execute("SELECT * FROM public.responsetimes")
print(cursor.fetchone())

df = pd.read_sql_query('SELECT * FROM public.responsetimes',con=conn)
df

DEBUG:__main__:b'SELECT * FROM public.responsetimes'


/tmp/ipykernel_305077/2219597650.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('SELECT * FROM public.responsetimes',con=conn)
DEBUG:__main__:b'SELECT * FROM public.responsetimes'


('http://worldtimeapi.org/api/timezone/Europe/Berlin', datetime.datetime(2024, 6, 6, 12, 34, 56, tzinfo=datetime.timezone.utc), 54782)


,url,measuretime,responsetime
0,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-06 12:34:56+00:00,54782
1,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:13:50.324863+00:00,0
2,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.104013+00:00,0
3,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.147828+00:00,0
4,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.190904+00:00,0
5,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.230391+00:00,0
6,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.271936+00:00,0
7,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.336191+00:00,0
8,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.380353+00:00,0
9,http://worldtimeapi.org/api/timezone/Europe/Be...,2024-06-08 22:15:46.432550+00:00,0


Now let's add data to the same database. First let's define again our response time function and use it, to generate some data.

In [10]:
from datetime import datetime
import requests
url = "http://google.com"

def measureResponseTimes(url, attempts):
    timeticks = []
    values = []
    result = {}

    for i in range(attempts):
        response = requests.post(url)
        timeticks.append(datetime.now())
        values.append(response.elapsed.total_seconds()) 
    result["timeticks"] = timeticks
    result["values"] = values
    return result

result = measureResponseTimes(url, 10)

In [20]:
# Convert data to a pandas DataFrame
df = pd.DataFrame(result)

# Add a column for URL (assuming the URL is the same for all records)
df['url'] = url  # Replace with the actual URL or remove if not applicable

# Rename columns to match the table structure
df.rename(columns={'timeticks': 'measuretime', 'values': 'responsetime'}, inplace=True)

# Convert responsetime to BIGINT (nanoseconds)
df['responsetime'] = (df['responsetime'] * 1e9).astype(int)

# Insert data into the PostgreSQL table
df.to_sql('responsetimes', conn, if_exists='fail', index=False)

/tmp/ipykernel_305077/28548997.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('responsetimes', conn, if_exists='fail', index=False)
DEBUG:__main__:b"\n        SELECT\n            name\n        FROM\n            sqlite_master\n        WHERE\n            type IN ('table', 'view')\n            AND name=?;\n        "


DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': syntax error at or near ";"
LINE 8:             AND name=?;
                              ^
